<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/paths_Tapis.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Tapis Paths  📒
***How DesignSafe File Storage and Tapis Work Together***

by Silvia Mazzoni, DesignSafe, 2025

Tapis powers file access and job submission on DesignSafe. It provides a consistent interface to interact with **multiple storage systems** and **compute environments**, making it easier to manage data before, during, and after simulation workflows.

With Tapis, you can:

* **List, upload, download, move, and delete files** across storage systems
* **Stage input files** (e.g., move from long-term storage to a compute node)
* **Collect outputs automatically** and return them to Corral (*MyData*)
* Use the same scripting or automation tools across locations

Tapis acts as the **glue** between DesignSafe’s storage and compute environments, streamlining data movement and improving reproducibility.

In this notebook, we’ll assemble a JSON dictionary of your Tapis paths and save it to your MyData (**~/MyData/.tapis_user_paths.json**) so you can reference it from any script at any time, even in new Jupyter Sessions, or from elsewhere.

Make sure you have "Established Your System Credentials" first!

In [1]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


In [2]:
# Connect to Tapis
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-22T07:29:28+00:00
 Token expires in: 2:00:12.145495
-- LOG IN SUCCESSFUL! --


In [3]:
# Initialize Json dictionary
TapisPaths = {}

---

## The Two Parts of a Tapis Path

A Tapis path is conceptually:

```
tapis://<SYSTEM_ID>/<RELATIVE_PATH>
```

1. ***SYSTEM_ID*** — which storage system you’re targeting
   Examples:
   * *designsafe.storage.default* (MyData),
   * *designsafe.storage.community* (Community),
   * *cloud.data* (Work allocations),
   or a
   * project-scoped system (e.g., *project-<uuid>*)

3. ***RELATIVE_PATH*** — the location **inside that system’s root**
   This is *not* relative to your program’s current directory; it’s relative to the **system root**.

   Tapis ignores CWD entirely; it’s always “(system, path-within-that-system).”


When building your tapis paths, it's always a good idea to define these two components separately and join them only in the input. This makes your script more portable and reusable.

---

## How Tapis Paths Differ from Jupyter Paths

* **Jupyter/OS paths** (e.g., */home/jovyan/...* or *data/run1.csv*) resolve relative to your **current working directory (CWD)** on that machine.
* **Tapis paths** ignore CWD. They always mean **(system, path-within-that-system)**, no matter where your code runs.
* For batch/automation (Tapis jobs, SLURM), Tapis paths are **more portable** than CWD-dependent relative paths.

---
## Obtaining File-System Paths

### Start with the Easy, Fixed Bases

These don’t depend on an HPC system or allocation. Once you know your username, the bases are stable. Because you can obtain your username programmatically from Tapis, you may not need to know it.

| Storage       | Typical Base (Tapis)                             | Notes                                  |
| ------------- | ------------------------------------------------ | -------------------------------------- |
| **MyData**    | `tapis://designsafe.storage.default/<username>/` | Your personal storage (aka Corral)     |
| **Community** | `tapis://designsafe.storage.community/`          | Public community content (read-mostly) |
| **Published** | `tapis://designsafe.storage.published/`          | Published content (read-only) |

**Examples**


* **MyData**:
  
    MyData/inputs/model.tcl
    → tapis://designsafe.storage.default/username/inputs/model.tcl
  
    ***don't forget your username!***


* **CommunityData**:
  
    CommunityData/Records/ATC-63/groundmotion.at2
    → tapis://designsafe.storage.community/Records/ATC-63/groundmotion.at2

* **Published**:
   
    Published/Records/ATC-63/groundmotion.at2
    → tapis://designsafe.storage.published/Records/ATC-63/groundmotion.at2

  You can find the relative path in the Data Depot


Use these as **bases**, then append project/job-specific subpaths.


#### Obtain your username programmatically
Using a utility function

In [4]:
username = OpsUtils.get_tapis_username(t)
print('username:',username)

username: silvia


In [5]:
# we will make the keys lower case, as they'll be easier to match
TapisPaths['mydata'] = f'tapis://designsafe.storage.default/{username}'
TapisPaths['community'] = f'tapis://designsafe.storage.community'
TapisPaths['published'] = f'tapis://designsafe.storage.published'

--

### Next: Work (User & System Dependent)

**Work** is the shared, high-performance project area mounted on both **JupyterHub** and **HPC**—ideal for staging inputs and storing outputs for jobs. Its base includes **allocation and username**, and differs by system (e.g., Stampede3 vs. LS6).

**Typical form:**

```
tapis://cloud.data/work/<allocation>/<username>/<system>/
```

Because this base is user/system-specific, it’s best to **discover it once** and **save it**.

### One-Time Setup (Recommended)

Use your utility function (e.g., `get_user_work_path`) to fetch and persist the base for each system you use (Stampede3, LS6, Frontera). Run it **once**, cache results to a small file (e.g., `~/.designsafe/user_paths.json`), and reuse forever. Because this app submits a job, it may take a bit to run.

In [6]:
from pathlib import Path
systems = ["stampede3", "ls6", "frontera"]

TapisPaths['work'] = {}
for s in systems:
    work_base = OpsUtils.get_user_work_tapis_uri(t, system_id=s)    
    print(f'{s}: {work_base}')
    TapisPaths['work'][s] = work_base

print('TapisPaths',TapisPaths)

stampede3: tapis://cloud.data/work/05072/silvia/stampede3/
ls6: tapis://cloud.data/work/05072/silvia/ls6/
frontera: tapis://cloud.data/work/05072/silvia/frontera/
TapisPaths {'mydata': 'tapis://designsafe.storage.default/silvia', 'community': 'tapis://designsafe.storage.community', 'published': 'tapis://designsafe.storage.published', 'work': {'stampede3': 'tapis://cloud.data/work/05072/silvia/stampede3/', 'ls6': 'tapis://cloud.data/work/05072/silvia/ls6/', 'frontera': 'tapis://cloud.data/work/05072/silvia/frontera/'}}


## Semi-Automatic: Copy from the Web Portal

If you don’t have a base yet (or you’re exploring):

1. Open an app page (e.g., OpenSeesMP on Stampede3).
2. Click the **folder** icon to browse.
3. Navigate to the target directory and select it.
4. Copy the displayed Tapis URI, e.g.

   ```
   tapis://cloud.data/work/05072/jdoe/stampede3/somefolder
   ```
5. Keep the **base portion** for reuse:

   ```
   tapis://cloud.data/work/05072/jdoe/stampede3/
   ```

Once you get the hang of this, it’s quick—then move that base into your saved JSON so you never have to browse again.

---

## (Optional) Manual Construction

When you know the patterns, you can write URIs by hand:

* **MyData**: `tapis://designsafe.storage.default/<username>/…`
* **Community**: `tapis://designsafe.storage.community/…`
* **Work**: `tapis://cloud.data/work/<allocation>/<username>/<system>/…`
* **Projects**: `tapis://project-<uuid-or-alias>/…` (project-scoped systems)

> These patterns include **site-specific pieces** (allocation numbers, project IDs). Grab them once (utility or portal), then store and reuse.

---

## Quick Reminders

* Tapis paths are **system-rooted**, not CWD-rooted (different from Jupyter).
* Use **Work** for HPC I/O; copy anything you want to keep from **Scratch** into **Work** or **MyData**.
* Save your bases once; **append relative subpaths** in all your scripts and job submissions.


## save to a file
This is the default ("~/MyData/.tapis_user_paths.json") path that will be used in the utility functions (OpsUtils) used in this training module.

In [7]:
import json
Path("~/MyData/.tapis_user_paths.json").expanduser().write_text(json.dumps(TapisPaths, indent=2))

375

## Use Utility Function
We have put the above steps into a utility function: get_tapis_base_paths(t,system) that you can call at the beginning of your script.

In [8]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_user_path.py'])

In [9]:
thisPath = OpsUtils.get_user_path_tapis_uri(t,'Work','stampede3')
print('thisPath:',thisPath)

found paths file: /home/jupyter/MyData/.tapis_user_paths.json
thisPath: tapis://cloud.data/work/05072/silvia/stampede3/
